In [1]:
### Quantiacs Trading System Template
# This program may take more than 10 minutes
# import necessary Packages
import stockstats
from stockstats import StockDataFrame as sdf
import quantiacsToolbox
import numpy as np
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize

In [ ]:
#initials for testing the code

#reading file 
F_AD = pd.read_csv(r'/home/anshul/Desktop/Quantiacs/tickerData/F_AD.txt')

#getting features
CLOSE = np.array(F_AD.loc[:252-1, [' CLOSE']])
#plt.plot(CLOSE)
VOL = np.array(F_AD.loc[:252-1, [' VOL']])
#plt.plot(VOL)
OI = np.array(F_AD.loc[:252-1, [' VOL']])
#plt.plot(OI)
HIGH = np.array(F_AD.loc[:252-1, [' HIGH']])
#plt.plot(HIGH)
LOW = np.array(F_AD.loc[:252-1, [' LOW']])
#plt.plot(LOW)
OPEN = np.array(F_AD.loc[:252-1, [' OPEN']])
#StockDataFrame.retype(OPEN)
# stock = sdf.retype(pd.DataFrame(OPEN, ))
# stock.get('macd')

In [3]:
#code
lookback = CLOSE.shape[0]
gap=5
X = np.concatenate([CLOSE[i:i + gap] for i in range(lookback - gap)], axis=1).T
y = np.sign((CLOSE[gap:lookback] - CLOSE[gap - 1:lookback - 1]).T[0])
#print(y[y==0])
clf = svm.SVC()
clf.fit(X, y)
print(clf.predict(CLOSE[-gap:].T))
#plt.plot(y)
plt.plot(clf.predict(CLOSE[-gap:].T))

[1.]


In [90]:
#code -> vwap/close
#20 days
vol = np.nan_to_num(VOL[-20:]).reshape(-1,1)
close = np.nan_to_num(CLOSE[-20:]).reshape(-1,1)
# if alpha > 1:
#     pos=1
# elif alpha < 0.3:
#     pos=-1
# else:
#     pos=0
    
scaler = MinMaxScaler(feature_range=(-1,1))
data = scaler.fit_transform(vol)
print(data)

[[-0.20772443]
 [-0.34551148]
 [-0.60542797]
 [-0.78079332]
 [-0.92484342]
 [-0.79853862]
 [-0.32359081]
 [ 1.        ]
 [ 0.15240084]
 [-0.05427975]
 [-0.07098121]
 [-0.85908142]
 [-0.84446764]
 [-0.93736952]
 [-0.81210856]
 [-0.94780793]
 [-0.99060543]
 [-1.        ]
 [-0.92588727]
 [-0.99686848]]


/home/anshul/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [20]:
a = np.array([1,2,3,4]).reshape(-1,1)
b = np.array([1,2,3,4]).reshape(-1,1)
print(a,b)
print(np.multiply(a,b))

[[1]
 [2]
 [3]
 [4]] [[1]
 [2]
 [3]
 [4]]
[[ 1]
 [ 4]
 [ 9]
 [16]]


In [22]:
class myStrategy(object):
    def myTradingSystem(self,DATE, OPEN, HIGH, LOW, CLOSE, VOL, OI, P, R, RINFO, exposure, equity, settings):
        
        nMarkets = len(settings['markets'])
        dur = settings['duration']
        
        vec_vwap = np.zeros((1, nMarkets), dtype='float')
        vec_hlv = np.zeros((1, nMarkets), dtype='float')
        pos = np.zeros((1, nMarkets), dtype='float')
        for i in range(nMarkets):
            close = np.nan_to_num(CLOSE[-dur:, i]).reshape(-1,1)
            vol = np.nan_to_num(VOL[-dur:, i]).reshape(-1,1)
            vwap = np.sum(np.multiply(vol,close))/np.sum(vol)
            alpha = np.nan_to_num(close[-1] - vwap)
            alpha1 = np.nan_to_num(np.sum(vol)-VOL[-1, i])
            vec_vwap[0, i] = alpha
            vec_hlv[0, i] = alpha1
        #print(vec_vwap)
        
        #neutralising alphas
        scaler = MinMaxScaler(feature_range=(0,1))
        data = vec_vwap.reshape(-1,1)
        min_max_data = -1*scaler.fit_transform(data)
        data_alpha1 = scaler.fit_transform(vec_hlv.reshape(-1,1))
            
        #position
        #pos = min_max_data.T
        neg_scaler = MinMaxScaler(feature_range=(-1,1))
        pos = neg_scaler.fit_transform(-data_alpha1 + min_max_data).T
        print('pos:', pos)
        
        
        
#         mean = np.mean(vec_vwap)
#         #print('mean:', mean)
#         for i in range(nMarkets):
#             if vec_vwap[0,i] >= 1:
#                 pos[0, i]=-1
#             elif vec_vwap[0,i] <= 1:
#                 pos[0, i]=1
#             else:
#                 pos[0, i]=0


        return pos, settings


    def mySettings(self):
        """ Define your trading system settings here """

        settings = {}

        # Futures Contracts
#         settings['markets'] = ['CASH', 'F_AD', 'F_AE', 'F_AH', 'F_AX', 'F_BC', 'F_BG', 'F_BO', 'F_BP', 'F_C',  'F_CA',
#                                'F_CC', 'F_CD', 'F_CF', 'F_CL', 'F_CT', 'F_DL', 'F_DM', 'F_DT', 'F_DX', 'F_DZ', 'F_EB',
#                                'F_EC', 'F_ED', 'F_ES', 'F_F',  'F_FB', 'F_FC', 'F_FL', 'F_FM', 'F_FP', 'F_FV', 'F_FY',
#                                'F_GC', 'F_GD', 'F_GS', 'F_GX', 'F_HG', 'F_HO', 'F_HP', 'F_JY', 'F_KC', 'F_LB', 'F_LC',
#                                'F_LN', 'F_LQ', 'F_LR', 'F_LU', 'F_LX', 'F_MD', 'F_MP', 'F_ND', 'F_NG', 'F_NQ', 'F_NR',
#                                'F_NY', 'F_O',  'F_OJ', 'F_PA', 'F_PL', 'F_PQ', 'F_RB', 'F_RF', 'F_RP', 'F_RR', 'F_RU',
#                                'F_RY', 'F_S',  'F_SB', 'F_SF', 'F_SH', 'F_SI', 'F_SM', 'F_SS', 'F_SX', 'F_TR', 'F_TU',
#                                'F_TY', 'F_UB', 'F_US', 'F_UZ', 'F_VF', 'F_VT', 'F_VW', 'F_VX',  'F_W', 'F_XX', 'F_YM',
#                               'F_ZQ']
        #markets to  decide:
        #currency -> 'US-Dollar Index', 'euro_fx', 'jap_yen', 'safrica rand', 
        #agri -> 'corn', 'cotton', coffee, sugar, wheat, rice, 
        #index -> volatility index, bloomberg commodity index, russel200, dax, cac 
        #energy -> nat gas,brent crude, newcastle coal,
        #int rate -> 30 day fed, eurodollar
        #bond-> 2 yr tre, 10 yr, 30 yr, euro bond
        #metal -> gold, cu, 
        #stocks -> NA
        settings['markets'] = ['F_DX', 'F_EC', 'F_JY', 'F_TR', 
                                'F_C', 'F_CT', 'F_KC', 'F_SB', 'F_W',  'F_NR', 
                                'F_VX', 'F_AH', 'F_RU', 'F_AX', 'F_CA', 
                                'F_NG', 'F_BC', 'F_LQ',
                                'F_ZQ', 'F_ED', 
                                'F_TU', 'F_TY', 'F_US', 'F_DT',  
                                'F_GC', 'F_HG']
        settings['lookback'] = 252
        settings['budget'] = 10 ** 6
        settings['slippage'] = 0.05

        settings['duration'] = 20

        return settings

In [ ]:
result = quantiacsToolbox.runts(myStrategy)

copying data from cache
Evaluating Trading System <class '__main__.myStrategy'>


/home/anshul/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


pos: [[ 0.0674454   0.0674454   0.60278365  0.0674454  -0.08414844 -0.14241123
   0.30625453  0.06492062  0.04063783 -0.05163897  0.0674454   0.0674454
   0.0674454   0.0674454   0.0674454   1.          0.0674454   0.0674454
   0.0674454  -0.82895256 -0.03542839 -0.03978436 -1.          0.0674454
  -0.1330786  -0.13059298]]
pos: [[-0.03988143 -0.03988143  0.75872716 -0.03988143 -0.17586067 -0.27460785
   0.26549534 -0.04722911 -0.12021852 -0.18541397 -0.03988143 -0.03988143
  -0.03988143 -0.03988143 -0.03988143  1.         -0.03988143 -0.03988143
  -0.03988143 -1.         -0.06692557  0.14400308 -0.89319009 -0.03988143
  -0.29585658 -0.28606506]]
pos: [[ 0.14925594  0.14925594  0.91544401  0.14925594 -0.00976376 -0.28294298
   0.7459322   0.02910803  0.03551516 -0.00525832  0.14925594  0.14925594
   0.14925594  0.14925594  0.14925594  1.          0.14925594  0.14925594
   0.14925594 -0.91725951  0.06269618  0.19290178 -1.          0.14925594
  -0.26562763 -0.20003598]]
pos: [[ 0.135194

pos: [[ 0.75265092  0.75265092  1.          0.75265092  0.58116284 -0.44725144
   0.05194294  0.57679408  0.73031998  0.63824296  0.75265092  0.75265092
   0.75265092  0.75265092  0.75265092  0.65658229  0.75265092  0.75265092
   0.75265092 -0.12742922  0.81845673  0.47698131 -1.          0.75265092
   0.80995054  0.43793952]]
pos: [[ 0.65252508  0.65252508  1.          0.65252508  0.5004291  -0.49675021
  -0.55881188  0.49698487  0.6949832   0.5108947   0.65252508  0.65252508
   0.65252508  0.65252508  0.65252508  0.66436168  0.65252508  0.65252508
   0.65252508 -0.30360846  0.77638884  0.49724503 -1.          0.65252508
   0.748976    0.35443877]]
pos: [[ 0.64812865  0.64812865  1.          0.64812865  0.50736067 -0.65516395
  -0.59132657  0.56854475  0.69794615  0.47837548  0.64812865  0.64812865
   0.64812865  0.64812865  0.64812865  0.79284643  0.64812865  0.64812865
   0.64812865 -0.35900949  0.76556456  0.59696501 -1.          0.64812865
   0.88759491  0.10944669]]
pos: [[ 0.519

pos: [[ 0.83785427  0.83785427 -0.20723534  0.83785427  0.61067503  0.00441418
   1.          0.95146188  0.85346688  0.7943138   0.83785427  0.83785427
   0.83785427  0.83785427  0.83785427  0.93290943  0.83785427  0.83785427
   0.83785427 -0.03085443  0.7134256   0.28287701 -1.          0.83785427
   0.70051827  0.71084629]]
pos: [[ 0.70396677  0.70396677  0.26786329  0.70396677  0.54607607 -0.07053439
   1.          0.85539565  0.75672584  0.71880862  0.70396677  0.70396677
   0.70396677  0.70396677  0.70396677  0.77978643  0.70396677  0.70396677
   0.70396677 -0.13637521  0.56815416  0.21819471 -1.          0.70396677
   0.53136079  0.35051003]]
pos: [[ 0.60996596  0.60996596  0.65540336  0.60996596  0.41561149 -0.25609559
   0.9444899   1.          0.75212213  0.65533439  0.60996596  0.60996596
   0.60996596  0.60996596  0.60996596  0.71107825  0.60996596  0.60996596
   0.60996596 -0.20039508  0.64773668  0.36884655 -1.          0.60996596
   0.57709453  0.19153156]]
pos: [[ 0.391

...

pos: [[ 0.31892882  0.30075364  0.54391304  0.44786791  0.2426396   0.54964865
   0.57050969  0.15188127  0.43959773  0.4058933   0.45786255  0.44374982
   0.33689307  0.23531931  0.40266869  0.28816641 -0.42187576 -0.40659377
   0.46479963  0.06929914  0.13516543 -1.          0.47363098 -0.46688912
   1.          0.40254623]]
pos: [[ 0.28214592  0.41688477  0.74871071  0.45216     0.23732194  0.60885367
   0.63201269  0.16212554  0.40475777  0.38882831  0.64701996  0.46411008
   0.05744356 -0.3383067   0.32261892  0.1813058  -0.70857648 -0.48928212
   0.511037    0.08076716  0.16955363 -1.          0.67822475 -0.36870898
   1.          0.3194441 ]]
pos: [[ 0.12524752  0.56206597  0.65031917  0.67959645  0.18737524  0.56820615
   0.58714838  0.16651391  0.33734371  0.31906092  0.44800856  0.44162691
   0.51041233 -0.07153543  0.33839168  0.18359171 -0.58183915 -0.42498545
   0.47561509  0.05043343  0.14161438 -1.          0.62210942 -0.46721306
   1.          0.35724341]]
pos: [[ 0.320

pos: [[ 0.26556004  0.00554647  0.46404004  0.16781467  0.12770757  0.25559864
   0.88549048  0.50709212  0.24758275  0.20420925  0.35831436  0.23464343
  -0.26569038  0.03465249  0.16452922 -0.43435559 -0.54009148  1.
   0.26822671 -0.05179498 -0.02275399 -1.          0.28568679 -0.09369986
   0.55819599 -0.18355032]]
pos: [[ 0.15697345 -0.0019771   0.34688344  0.08716222  0.05887465  0.17163472
   0.74427062  0.33446061  0.17452916  0.14737408  0.28210306  0.16990789
  -0.27858713 -0.1465756   0.05430934 -0.32324804 -0.39148335  1.
   0.19407034 -0.10054308 -0.07557422 -1.          0.18652021 -0.12232477
   0.44092693 -0.11311597]]
pos: [[ 2.68482259e-01  5.77256853e-04  3.07372339e-01  1.44276092e-01
   1.32598732e-01  2.60302677e-01  7.17902078e-01  3.68480795e-01
   2.63316487e-01  2.37794831e-01  3.14704473e-01  2.57749430e-01
  -1.85853620e-01 -3.88275483e-01  8.32256838e-02 -1.23384948e-01
  -2.19277049e-01  1.00000000e+00  2.72678728e-01 -3.81998091e-03
  -2.18493151e-02 -1.00

pos: [[ 0.63157209 -0.25144484 -0.26778883  0.02745313  0.11481689 -0.18700341
   0.97609983  0.01848144  0.18586098  0.51888467  0.69905175  0.33244103
  -0.47998514 -0.07026646  0.4007615   1.         -0.42068687  0.16129377
   0.30811857  0.06300195  0.05833479 -1.          0.85443648 -0.4250341
  -0.55092249  0.6579172 ]]
pos: [[ 0.55018034  0.08160542 -0.2567593   0.28994094  0.25630756  0.21681044
   0.98182151  0.1977666   0.40087908  0.54837359  0.57782517  0.42059604
   0.22776323  1.          0.56332594  0.79871969  0.16309718  0.74462813
   0.38045583  0.16047588  0.11539957 -1.          0.27721039 -0.45527859
  -0.40606142  0.38863159]]
pos: [[ 0.37524557  0.22213125 -0.11801225  0.30775351  0.1663782   0.20776984
   0.8262196   0.1799284   0.25627528  0.49354837  0.47631726  0.36713987
   0.27786528  0.7474345   0.52816651  0.51149552  0.2908372   1.
   0.32984179  0.13293665  0.09472517 -1.          0.07803713 -0.49303386
  -0.39272701  0.455299  ]]
pos: [[ 0.3050487   0.

pos: [[ 0.69924445  0.58837634  0.34955808  1.          0.52472221  0.74385679
   0.77076354  0.86678647  0.69312646  0.65644527  0.62834835  0.7276974
   0.46073311 -0.12716801  0.5022034   0.56469251  0.16426358 -0.5598515
   0.73163535  0.3801169   0.36697142 -1.          0.17372521 -0.50105515
   0.12604578  0.78782465]]
pos: [[ 0.72970166  0.65437269  0.34326706  1.          0.5939338   0.78116167
   0.80971139  0.90804475  0.72932108  0.70124817  0.70744149  0.75436672
   0.50715524 -0.1509092   0.61247327  0.35143769 -0.00447236 -0.59464175
   0.77137647  0.42077424  0.40822044 -1.          0.18739386 -0.52777414
   0.08842566  0.80712921]]
pos: [[ 0.61484541  0.58329599  0.32291575  1.          0.47989621  0.67811606
   0.70133721  0.78987822  0.59963164  0.5720808   0.49431802  0.64900074
   0.76607797  0.07179719  0.53607242  0.32006266 -0.11274133 -0.51128509
   0.67784951  0.3402701   0.32295594 -1.          0.2182204  -0.57025335
   0.21165135  0.43191442]]
pos: [[ 0.55052

pos: [[ 0.67491783 -0.1170784   0.01167954  0.23827512  0.2960821   0.75347591
   0.99894055  0.79161181  0.4203098   0.32482734  0.5572512   0.49689646
  -0.19490194 -0.29382172  0.34793694  1.          0.46077557 -0.01835181
   0.47757415  0.20615675  0.09604918 -1.         -0.29032232 -0.32410711
  -0.30643686  0.303589  ]]
pos: [[ 0.62247925 -0.05346689  0.00724106  0.15964239  0.28111502  0.73339663
   0.94333813  0.75639458  0.40098416  0.34583358  0.5086019   0.48420259
   0.05498741 -0.32560724  0.38072121  1.          0.5393531  -0.23204528
   0.45992696  0.17374166  0.08390271 -1.         -0.16443785 -0.26982408
  -0.33630885  0.39342424]]
pos: [[ 0.71211372  0.02851032 -0.07788874  0.31820747  0.31150876  0.82062067
   1.          0.82576313  0.41432462  0.40321403  0.54799815  0.55471581
   0.06479715  0.38656298  0.52531608  0.97581762  0.50276443 -0.24616302
   0.53775955  0.24443194  0.13957278 -1.         -0.18928663 -0.25191714
  -0.49250001  0.53554884]]
pos: [[ 0.743